In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
files = 'instance/small case/IES_E9H12G7-v1.xlsx'
excel = pd.read_excel(files, sheet_name=None)

In [10]:
def interpolate_load(data: pd.DataFrame, len_multiply: int, save_path: str, save: bool = True) -> dict:
    """Interpolate load data to a new length.

    Args:
        data (pd.DataFrame): A dataframe that has 'wind', 'eLoad', 'hLoad', 'gLoad' as keys.
        len_multiply (int): amount of points in between each data point
    
    Returns:
        Saves the interpolated data to csv files in the specified save_path.
        It saves files for each key in data.
        returns a dictionary with the interpolated data.
    """
    if save_path[-4:] == '.csv':
        save_path = save_path[:-4]
    if save_path[-1] == '/':
        save_path = save_path[:-1]
    
    new_len = int(len(data['wind']) * len_multiply)

    excel = {}
    t = np.arange(1, new_len//2 + 1)
    time = np.concatenate((-t[::-1], t))

    changes = ['wind', 'eLoad', 'hLoad', 'gLoad']
    for key in changes:
        csv_dir = {}
        csv_dir['time'] = time
        for bus in data[key].keys()[1:]:
            dat = data[key][bus].values
            inter = np.full(new_len, np.nan)
            inter[::2] = dat
            csv_dir[bus] = inter
        
        new_excel = pd.DataFrame(csv_dir).interpolate(method='polynomial', order=2).fillna(method='ffill')
        if save == True:
            new_excel.to_csv(save_path+'/'+key+'.csv', index=False)

        excel[key] = new_excel

        
    
    return excel

def interpolated_excel(data: pd.DataFrame, len_multiply: int, save_path: str, filename: str) -> None:
    """
    Interpolates the data and saves it to an excel file.

    Args:
        data (pd.DataFrame): A dataframe that consists of entire excel file.
        len_multiply (int): amount of points in between each data point
        save_path (str): path to save the excel file
        filename (str): name of the excel file
    """
    if save_path[-1] != '/':
        save_path += '/'
    if filename[-5:] != '.xlsx':
        filename += '.xlsx'
    loads = interpolate_load(data, len_multiply, save_path, save=False)
    for key in loads.keys():
        data[key] = pd.DataFrame(loads[key])
    
    with pd.ExcelWriter(save_path+filename) as writer:
        for sheet_name in excel.keys():
            excel[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)



In [11]:
interpolated_excel(excel, 2, 'instance/small case', 'v3.xlsx')